In [9]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, regexp_extract, row_number, desc

StatementMeta(, 21b36222-8753-484a-896f-bbda78b4c92b, 11, Finished, Available, Finished)

In [10]:
llm_df = spark.read.table("LLMResponses")
conn_df = spark.read.table("Connections")

StatementMeta(, 21b36222-8753-484a-896f-bbda78b4c92b, 12, Finished, Available, Finished)

In [11]:
# Deduplicate Connections: keep earliest non-deleted session event per session_id
window_spec = Window.partitionBy("session_id").orderBy(desc("event_timestamp"))
conn_dedup = (conn_df.filter(col("is_deleted") == 0)
                     .withColumn("rn", row_number().over(window_spec))
                     .filter(col("rn") == 1)
                     .drop("rn"))

StatementMeta(, 21b36222-8753-484a-896f-bbda78b4c92b, 13, Finished, Available, Finished)

In [12]:
# Extract scenario and customer from prompt (example regex placeholders)
# Adjust regex patterns based on actual prompt format
llm_extracted = llm_df.withColumn("scenario", regexp_extract(col("prompt"), r'Scenario:\s*(\w+)', 1)) \
                     .withColumn("customer", regexp_extract(col("prompt"), r'Customer:\s*([\w\s]+)', 1))

StatementMeta(, 21b36222-8753-484a-896f-bbda78b4c92b, 14, Finished, Available, Finished)

In [13]:
# Join LLM responses with user sessions
joined_df = llm_extracted.join(conn_dedup, on="session_id", how="left")

StatementMeta(, 21b36222-8753-484a-896f-bbda78b4c92b, 15, Finished, Available, Finished)

In [14]:
# Aggregate most recommended databases per scenario
# Assuming databases are mentioned in response_text, extract database names (example)
joined_df = joined_df.withColumn("recommended_db", regexp_extract(col("response_text"), r'(Azure SQL DB|Azure PostgreSQL|Azure Cosmos DB)', 1))

agg_df = (joined_df.groupBy("scenario", "recommended_db")
                 .count()
                 .orderBy(desc("count")))

agg_df.show()

StatementMeta(, 21b36222-8753-484a-896f-bbda78b4c92b, 16, Finished, Available, Finished)

+-----------+----------------+-----+
|   scenario|  recommended_db|count|
+-----------+----------------+-----+
|           | Azure Cosmos DB|  368|
|       NULL|                |   82|
|           |                |   56|
|       NULL| Azure Cosmos DB|   55|
|           |Azure PostgreSQL|   46|
|           |    Azure SQL DB|   29|
|       NULL|Azure PostgreSQL|   12|
|       Chat|                |    4|
|  Knowledge| Azure Cosmos DB|    3|
|  Knowledge|                |    3|
|       Chat| Azure Cosmos DB|    3|
|       NULL|    Azure SQL DB|    1|
|Modernizing|    Azure SQL DB|    1|
+-----------+----------------+-----+



In [15]:
# Write gold table
agg_df.write.format("delta").mode("overwrite").saveAsTable("agg_databases_choices")

StatementMeta(, 21b36222-8753-484a-896f-bbda78b4c92b, 17, Finished, Available, Finished)